In [2]:
import httplib2
from BeautifulSoup import BeautifulSoup, SoupStrainer
from pprint import pprint
import json

class Link():
    def getName(self):
       return self.name

    def getWatched(self):
        return self.watched
    
    def setWatched(self):
        self.watched = True 
    
    def getDict(self):
        return self.dict
    
    def addToDict(self, where, how):
        self.dict[where] = how;
    
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=True, indent=4)

    def __init__(self, name, title, h1, description, where, how):
       self.name = name
       self.title = title
       self.h1 = h1
       self.description = description
       self.watched = False 
       self.dict = dict()
       self.dict[where] = how;

def getFirstNotWatchedLink(dict):
    for x in dict:
        if (not dict[x].getWatched()):
            return x
    return None

def addNewLink(name, title, h1,  description, where, how):
    try:
        links[name].addToDict(where, how)
    except KeyError, e:
        object = Link(name, title, h1, description,  where, how) 
        links[object.getName()] = object

In [7]:
http = httplib2.Http()
base_url = 'http://localhost:9000'
cur_url = '/'
host = "localhost"
links = dict() 
title = ""
h1 = ""
description = ""

addNewLink("/", "/", "/", "/", "/", "/")


while getFirstNotWatchedLink(links):
    cur_url = getFirstNotWatchedLink(links)
    status, response = http.request(base_url+cur_url)
        
    soup = BeautifulSoup(response)
    title = soup.title.string
    h1 = soup.find('h1')
    if (h1):
        h1 = h1.string
    description = soup.findAll(attrs={"name":"description"}) 
    if (description):
        try:
            description = description[0]['content'].encode('utf-8')
        except KeyError, e:
            description = ""
        
    for link in BeautifulSoup(response, parseOnlyThese=SoupStrainer('a')):
        if ( link.get("href") and not link.get("href").startswith('#') and not link.get("href").startswith('http') and not link.get("href").startswith('//') and not link.get("href").startswith('tel') and not link.get("href").startswith('mailto') ):
            str = link.string
            addNewLink(link.get("href"), title, h1, description, cur_url, str )
    links[cur_url].setWatched()

file = open(host+'.js',"w") 
 
file.write("var data = [")
for x in links:
        file.write(links[x].toJSON()) 
        file.write(",")
file.write("]")
file.close() 

    
    
